<a href="https://colab.research.google.com/github/jacksonStone/sentiment-analysis-with-hot-one-encoding-2/blob/main/DataGeneratorExperiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00


First we need to generate a test data set.
I leverage Grok and the latest Llama 3 to do this.

In [ ]:
from groq import Groq
from google.colab import userdata
from time import sleep

client = Groq(
    api_key=userdata.get("GROQ_API_KEY"),
)
betterPositiveCompletions = []
for j in range(30):
  if(j > 0):
    sleep(50)
  for i in range(30):
    chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": "Generate 10 random positive statements. Be succient. One sentence per statement. Go straight to the content with no preamble. Structure them like so: STATEMENT:some positive statement.\nSTATEMENT:some other positive statement\netc.",
          }
      ],
      model="llama3-70b-8192",
    )
    betterPositiveCompletions.append(chat_completion.choices[0].message.content)
  print("finished batch: " + str(j) + " out of 100")
print(betterPositiveCompletions)
# persist chatCompletions to pickle
parsedPositiveCompletions = []
for i in betterPositiveCompletions:
  parsedPositiveCompletions += i.split("\nSTATEMENT:")[1:]
print(parsedPositiveCompletions)

betterNegativeCompletions = []
for j in range(30):
  if(j > 0):
    sleep(50)
  for i in range(30):
    chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": "Generate 10 random negative statements. Be succient. One sentence per statement. Go straight to the content with no preamble. Structure them like so: STATEMENT:some negative statement.\nSTATEMENT:some other negative statement\netc.",
          }
      ],
      model="llama3-70b-8192",
    )
    betterNegativeCompletions.append(chat_completion.choices[0].message.content)
  print("finished batch: " + str(j) + " out of 100")
print(betterNegativeCompletions)
# persist chatCompletions to pickle
parsedNegativeCompletions = []
for i in betterNegativeCompletions:
  parsedNegativeCompletions += i.split("\nSTATEMENT:")[1:]
print(parsedNegativeCompletions)

# persist chatCompletions to pickle
import pickle
with open('negativeChats2.pkl', 'wb') as f:
    pickle.dump(parsedNegativeCompletions, f)



finished batch: 0 out of 100
finished batch: 1 out of 100
finished batch: 2 out of 100
finished batch: 3 out of 100
finished batch: 4 out of 100
finished batch: 5 out of 100
finished batch: 6 out of 100
finished batch: 7 out of 100
finished batch: 8 out of 100
finished batch: 9 out of 100
finished batch: 10 out of 100
finished batch: 11 out of 100
finished batch: 12 out of 100
finished batch: 13 out of 100
finished batch: 14 out of 100
finished batch: 15 out of 100
finished batch: 16 out of 100
finished batch: 17 out of 100
finished batch: 18 out of 100
finished batch: 19 out of 100
finished batch: 20 out of 100
finished batch: 21 out of 100
finished batch: 22 out of 100
finished batch: 23 out of 100
finished batch: 24 out of 100
finished batch: 25 out of 100
finished batch: 26 out of 100
finished batch: 27 out of 100
finished batch: 28 out of 100
finished batch: 29 out of 100
["Here are 10 random positive statements:\n\nSTATEMENT:Believing in yourself is the first step towards achievi

Then we save the data sets to Drive to prevent needing to rerun the 20min script

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
with open('/content/drive/MyDrive/negativeChats.pkl', 'wb') as f:
    pickle.dump(parsedNegativeCompletions, f)

In [4]:
with open('/content/drive/MyDrive/positiveChats.pkl', 'wb') as f:
    pickle.dump(parsedPositiveCompletions, f)

Now I need to convert the data into numerical values

In [5]:
import string
import pickle
from random import shuffle
positiveChats = []
negativeChats = []
with open('/content/drive/MyDrive/positiveChats.pkl', 'rb') as f:
    positiveChats = dict.fromkeys(pickle.load(f)).keys()
with open('/content/drive/MyDrive/negativeChats.pkl', 'rb') as f:
    negativeChats = dict.fromkeys(pickle.load(f)).keys()
print("Positive chats: " + str(len(positiveChats)))
print("Negative chats: " + str(len(negativeChats)))
labeledPositiveChats = []
labeledNegativeChats = []
for chat in positiveChats:
  labeledPositiveChats.append((chat, "positive"))
for chat in negativeChats:
  labeledNegativeChats.append((chat, "negative"))
allChats = labeledPositiveChats + labeledNegativeChats
shuffle(allChats)
wordIndex = {}
counter = {"i": 0}
def convert_string_to_numbers(chat):
  numberifiedChat = []
  for word in chat.strip().split():
    word = word.lower()
    punctuation = ""
    if word[-1] in string.punctuation:
      punctuation = word[-1]
      if punctuation not in wordIndex:
        wordIndex[punctuation] = [counter["i"], 1]
        counter["i"] = counter["i"] + 1
      else:
        wordIndex[punctuation][1] += 1
      word = word[:-1]
    if len(word) > 0:
      if word not in wordIndex:
        wordIndex[word] = [counter["i"], 1]
        counter["i"] = counter["i"] + 1
      else:
        wordIndex[word][1] += 1
      numberifiedChat.append(wordIndex[word][0])
      if punctuation != "":
        numberifiedChat.append(wordIndex[punctuation][0])
  return numberifiedChat

print(len(allChats))
#filter out duplicative chats
chatsConvertedToNumbers = []
for (chat, sentiment) in allChats:
  numberifiedChat = convert_string_to_numbers(chat)
  if(len(numberifiedChat) < 21):
    chatsConvertedToNumbers.append((numberifiedChat, sentiment))
wordCount  = len(wordIndex.keys())
print("word count: " + str(wordCount))
print("Cleaned up test sentences: " + str(len(chatsConvertedToNumbers)))
print(allChats[:10])

Positive chats: 3995
Negative chats: 5812
9807
word count: 3323
Cleaned up test sentences: 9737
[('Every experience, no matter how big or small, shapes you into a stronger person.', 'positive'), (' The new hire is completely unqualified for the job.', 'negative'), ('Every day is a new chance to start fresh and make a positive impact.', 'positive'), ("Your smile has the power to brighten someone's day and make the world a happier place.", 'positive'), ('You are capable of achieving your wildest dreams.', 'positive'), ("I'll never find a job in this tough market.", 'negative'), ('Every challenge you face is a chance to learn and become better.', 'positive'), ("I'm not smart enough to succeed in this world.\n", 'negative'), ('Every day is a new opportunity to start again and make a change for the better.', 'positive'), ('Today is a brand new day full of new possibilities.', 'positive')]


In [6]:
# sort wordIndex by the second entry in each array in the 2d array
sortedWordIndex = sorted(wordIndex.items(), key=lambda x: x[1][1], reverse=True)
print(sortedWordIndex)
#substitute allChats first tuple entry with an array of numbers using the index value
for (chat, _) in allChats:
  for word in chat.strip().strip("\n").split():
    word = word.lower()


[('.', [14, 9625]), ('and', [29, 5171]), ('the', [16, 3846]), ('to', [26, 3843]), ('is', [19, 3840]), ('a', [12, 3674]), (',', [1, 3362]), ('you', [10, 2717]), ('your', [33, 2025]), ('of', [44, 1936]), ('are', [42, 1719]), ('in', [51, 1586]), ('my', [95, 1462]), ('every', [0, 1314]), ('life', [124, 990]), ('new', [17, 986]), ('no', [3, 934]), ("i'm", [60, 906]), ('this', [52, 874]), ('be', [180, 851]), ('make', [30, 757]), ('never', [49, 756]), ('i', [142, 693]), ('day', [24, 651]), ('has', [35, 638]), ('will', [119, 619]), ('me', [101, 588]), ('for', [22, 577]), ('world', [39, 570]), ('with', [91, 553]), ('can', [171, 528]), ('just', [260, 521]), ("i'll", [48, 478]), ('start', [27, 475]), ('matter', [4, 464]), ('not', [61, 456]), ('how', [5, 418]), ('always', [120, 405]), ('good', [272, 398]), ('it', [89, 397]), ('today', [68, 392]), ('so', [149, 388]), ('enough', [63, 384]), ('opportunity', [65, 381]), ('that', [183, 380]), ('from', [83, 378]), ('have', [186, 368]), ('all', [97, 365]

In [7]:
# check if there are any repeats in allChats
for i in range(len(allChats)):
  for j in range(i+1, len(allChats)):
    if allChats[i][0] == allChats[j][0]:
      print(allChats[i])

I covert from From word-indexes to one-hot-vectors. The theory is each word has a unique meaning that is unknown to my model. This seemed better than the rather brittle embedding approaches I was seening in the literature.

In [8]:
import numpy as np
chats_to_hot_one_formats = np.zeros(shape=(len(chatsConvertedToNumbers),20,wordCount+50), dtype="uint8")
count = 0
for i in chatsConvertedToNumbers:
  two_d_array = chats_to_hot_one_formats[count]
  count += 1
  word = 0
  for wordNumber in i[0]:
    two_d_array[word][wordNumber] = 1
    word += 1
print(chats_to_hot_one_formats)

def convert_string_to_hot_one(chat):
  hot_one_array = np.zeros(shape=(20,wordCount+50), dtype="uint8")
  word = 0

  numberified = convert_string_to_numbers(chat)
  for wordNumber in numberified:
    hot_one_array[word][wordNumber] = 1
    word += 1
  return hot_one_array

[[[1 0 0 ... 0 0 0]
  [0 0 1 ... 0 0 0]
  [0 1 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[1 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]


Now for Defining the model.

In [9]:
!pip install torch
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define the neural network model
class BinaryClassificationModel(nn.Module):
    def __init__(self):
        super(BinaryClassificationModel, self).__init__()
        self.layers = nn.ModuleList()

        # Input layer
        self.layers.append(nn.Linear((wordCount+50) * 20, 20))

        # Repeating dense layers
        for _ in range(1):
            self.layers.append(nn.Linear(20, 20))

        # Output layer
        self.output_layer = nn.Linear(20, 1)

    def forward(self, x):
        x = x.view(-1, (wordCount+50) * 20)  # Flatten the input
        for layer in self.layers:
            x = torch.relu(layer(x))
        x = torch.sigmoid(self.output_layer(x))
        return x

# Instantiate the model
model = BinaryClassificationModel()

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Example function to train the model on one batch
def train_model(model, criterion, optimizer, test_data, test_labels, val_data, val_labels, epochs=20):
    for epoch in range(epochs):
        model.train()

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(test_data)
        loss = criterion(outputs, test_labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # check against validation set
        model.eval()
        with torch.no_grad():
            val_outputs = model(val_data)
            val_loss = criterion(val_outputs, val_labels)
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Validation Loss: {val_loss.item():.4f}')

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

Put labels in the correct format

In [10]:
labelOfChats = []
for (chat, sentiment) in chatsConvertedToNumbers:
  if sentiment == "positive":
    labelOfChats.append([1])
  else:
    labelOfChats.append([0])
# Example labels (replace with your actual labels)
labels = torch.tensor(np.array(labelOfChats), dtype=torch.float32)  # Convert to a PyTorch tensor


Divide test and validation set, about 50/50 seemed good

In [11]:

# Convert NumPy matrices to a PyTorch tensor
data_tensor = torch.tensor(chats_to_hot_one_formats, dtype=torch.float32)  # Convert to a PyTorch tensor
data_tensor = data_tensor.view(len(chats_to_hot_one_formats), -1)  # Reshape to (num_samples, 2500 * 20)

# split data_tensor to leave off the last 1000 elements
division = -3000
test_data = data_tensor[:division]
test_labels = labels[:division]
val_data = data_tensor[division:]
val_labels = labels[division:]
# Train the model
train_model(model, criterion, optimizer, test_data, test_labels, val_data, val_labels)

Epoch [1/20], Loss: 0.6934, Validation Loss: 0.6721
Epoch [2/20], Loss: 0.6703, Validation Loss: 0.6394
Epoch [3/20], Loss: 0.6351, Validation Loss: 0.5973
Epoch [4/20], Loss: 0.5898, Validation Loss: 0.5472
Epoch [5/20], Loss: 0.5353, Validation Loss: 0.4894
Epoch [6/20], Loss: 0.4725, Validation Loss: 0.4240
Epoch [7/20], Loss: 0.4020, Validation Loss: 0.3537
Epoch [8/20], Loss: 0.3274, Validation Loss: 0.2826
Epoch [9/20], Loss: 0.2533, Validation Loss: 0.2154
Epoch [10/20], Loss: 0.1848, Validation Loss: 0.1568
Epoch [11/20], Loss: 0.1266, Validation Loss: 0.1100
Epoch [12/20], Loss: 0.0815, Validation Loss: 0.0759
Epoch [13/20], Loss: 0.0496, Validation Loss: 0.0530
Epoch [14/20], Loss: 0.0291, Validation Loss: 0.0386
Epoch [15/20], Loss: 0.0167, Validation Loss: 0.0300
Epoch [16/20], Loss: 0.0097, Validation Loss: 0.0250
Epoch [17/20], Loss: 0.0058, Validation Loss: 0.0222
Epoch [18/20], Loss: 0.0036, Validation Loss: 0.0206
Epoch [19/20], Loss: 0.0025, Validation Loss: 0.0198
Ep

Now I can spot check values in the model


In [13]:
# test the model against a new string
output = model(torch.tensor(convert_string_to_hot_one("You are not nice."), dtype=torch.float32).view(1, -1))
output = output.detach().numpy()
if(output[0][0] > 0.5):
  print("positive")
else:
  print("negative")

negative
